In [10]:
import pandas as pd
import requests
import json
import time
import grequests
from config import api_key
import numpy as np

In [75]:
movies_dfs = []
for year in range(1982,2020):
    for month in range(1,12):
            if year == 2019 and month == 4:
                break
            one_df = pd.read_html(f"https://www.boxofficemojo.com/monthly/?view=releasedate&chart=&month={month}&yr={year}", header=0)[1]
            one_df = one_df.iloc[:-4]
            one_df = one_df.rename(columns={ one_df.columns[2]: "Studio", "Total Gross / Theaters": "Total Gross", 
                                        "Total Gross / Theaters.1": "TG Theaters", "Opening / Theaters" : "Opening", 
                                        "Opening / Theaters.1": "O Theaters", "Movie Title (click to view)": "Movie Title"})
            one_df["Year"] = year
            movies_dfs.append(one_df)
movie_df = pd.concat(movies_dfs,sort = True)

movie_df


Close                              Movie Title O Theaters  Open  \
0    NaN                               Vice Squad        510  1/22   
1    NaN                            The Seduction        597  1/22   
2    NaN                           Shoot the Moon        585  1/22   
3    NaN                               The Border         84  1/29   
4    NaN                                    Venom        900  1/29   
5    NaN                   A Stranger is Watching        NaN  1/22   
6    NaN                              I, the Jury        NaN   1/9   
0    NaN                           Quest for Fire        223  2/12   
1    NaN                            Death Wish II        377  2/19   
2    NaN                                  Missing        733  2/12   
3    NaN                              Making Love        363  2/12   
4    NaN                                 Das Boot          2  2/10   
5    NaN                           Night Crossing        640   2/5   
6    NaN                         The Beast Within        550  2/12   
7    NaN                              The Amateur        437  2/12   
8    NaN                            Personal Best        142   2/5   
9    NaN                              Cannery Row        700  2/12   
10   NaN                                Barbarosa        187  2/19   
11   NaN                       One from the Heart         41  2/12   
12   NaN                           Love and Money        NaN  2/12   
0    NaN                                  Porky's       1148  3/19   
1    NaN  Richard Pryor: Live on the Sunset Strip       1277  3/12   
2    NaN                          Victor/Victoria          3  3/19   
3    NaN                                Deathtrap        341  3/19   
4    NaN                                    Diner         21   3/5   
5    NaN                I Ought to Be in Pictures        662  3/26   
6    NaN                       Evil Under the Sun        609   3/5   
7    NaN              Weavers: Wasn't That a Time        NaN   3/3   
8    NaN             I'm Dancing as Fast as I Can        NaN   3/5   
0    NaN               The Sword and the Sorcerer        233  4/23   
..   ...                                      ...        ...   ...   
57     -                                  3 Faces          1   3/8   
58     -                               Ramen Shop          2  3/22   
59     -                       Slut in a Good Way          7  3/28   
60     -                             Lost & Found          6  3/29   
61     -                                   Yardie         77  3/15   
62     -                                The Sower          2   3/1   
63     -                An Elephant Sitting Still          1   3/8   
64     -                   He Matado a mi Marido!         11   3/1   
65     -                            Working Woman          2  3/27   
66     -                              Knife+Heart          2  3/15   
67     -                              Maze (2019)          2  3/22   
68   3/7                   The Hole in the Ground        NaN   3/1   
69     -                              Out of Blue         35  3/22   
70     -     Joseph Pulitzer: Voice of the People          1   3/1   
71     -                             Buddy (2019)        NaN  3/20   
72     -                             Black Mother        NaN   3/9   
73     -                       Virginia Minnesota          2   3/1   
74     -                           Combat Obscura         10  3/15   
75     -                                Screwball         13  3/29   
76     -                           Ferrante Fever          1   3/8   
77     -                               Off Season          2  3/15   
78     -                 The Eyes of Orson Welles          1  3/15   
79     -                          Zen for Nothing          2  3/29   
80     -                                 Starfish        NaN  3/15   
81     -                   This Magnificent Cake!          1   3/1   
82     -     

In [ ]:
movie_df.to_csv("Movie_Data_Initial.csv")

In [76]:
movie_df["Movie Title"] = movie_df["Movie Title"].str.replace(r"\(.*\)","")

movie_df["Title"] = ""
movie_df["Rated"] = ""
movie_df["Released"] = ""
movie_df["Runtime"] = ""
movie_df["Genre"] = ""
movie_df["Director"] = ""
movie_df["Writer"] = ""
movie_df["Actors"] = ""
movie_df["Plot"] = ""
movie_df["Language"] = ""
movie_df["Country"] = ""
movie_df["Awards"] = ""
movie_df["Poster"] = ""
movie_df["Ratings"] = ""
movie_df["Metascore"] = ""
movie_df["imdbRating"] = ""
movie_df["imdbVotes"] = ""
movie_df["imdbID"] = ""
movie_df["DVD"] = ""
movie_df["BoxOffice"] = ""
movie_df["Production"] = ""
movie_df["Website"] = ""

movie_df = movie_df.reset_index(drop=True)
movie_df

Close                              Movie Title O Theaters  Open  \
0       NaN                               Vice Squad        510  1/22   
1       NaN                            The Seduction        597  1/22   
2       NaN                           Shoot the Moon        585  1/22   
3       NaN                               The Border         84  1/29   
4       NaN                                    Venom        900  1/29   
5       NaN                   A Stranger is Watching        NaN  1/22   
6       NaN                              I, the Jury        NaN   1/9   
7       NaN                           Quest for Fire        223  2/12   
8       NaN                            Death Wish II        377  2/19   
9       NaN                                  Missing        733  2/12   
10      NaN                              Making Love        363  2/12   
11      NaN                                 Das Boot          2  2/10   
12      NaN                           Night Crossing        640   2/5   
13      NaN                         The Beast Within        550  2/12   
14      NaN                              The Amateur        437  2/12   
15      NaN                            Personal Best        142   2/5   
16      NaN                              Cannery Row        700  2/12   
17      NaN                                Barbarosa        187  2/19   
18      NaN                       One from the Heart         41  2/12   
19      NaN                           Love and Money        NaN  2/12   
20      NaN                                  Porky's       1148  3/19   
21      NaN  Richard Pryor: Live on the Sunset Strip       1277  3/12   
22      NaN                          Victor/Victoria          3  3/19   
23      NaN                                Deathtrap        341  3/19   
24      NaN                                    Diner         21   3/5   
25      NaN                I Ought to Be in Pictures        662  3/26   
26      NaN                       Evil Under the Sun        609   3/5   
27      NaN              Weavers: Wasn't That a Time        NaN   3/3   
28      NaN             I'm Dancing as Fast as I Can        NaN   3/5   
29      NaN               The Sword and the Sorcerer        233  4/23   
...     ...                                      ...        ...   ...   
14764     -                                  3 Faces          1   3/8   
14765     -                               Ramen Shop          2  3/22   
14766     -                       Slut in a Good Way          7  3/28   
14767     -                             Lost & Found          6  3/29   
14768     -                                   Yardie         77  3/15   
14769     -                                The Sower          2   3/1   
14770     -                An Elephant Sitting Still          1   3/8   
14771     -                   He Matado a mi Marido!         11   3/1   
14772     -                            Working Woman          2  3/27   
14773     -                              Knife+Heart          2  3/15   
14774     -                                    Maze           2  3/22   
14775   3/7                   The Hole in the Ground        NaN   3/1   
14776     -                              Out of Blue         35  3/22   
14777     -     Joseph Pulitzer: Voice of the People          1   3/1   
14778     -                                   Buddy         NaN  3/20   
14779     -                             Black Mother        NaN   3/9   
14780     -                       Virginia Minnesota          2   3/1   
14781     -                           Combat Obscura         10  3/15   
14782     -                                Screwball         13  3/29   
14783     -                           Ferrante Fever          1   3/8   
14784     -                               Off Season          2  3/15   
14785     -                 The Eyes of Orson Welles          1  3/15   
14786     -                          Zen for Nothing          2 

In [ ]:
dups = pd.concat(g for _, g in movie_df.groupby("Movie Title") if len(g) > 1)
movie_df
dups

In [93]:
# pd.set_option('display.max_columns', None) 
def appendData(reqs):
    for index, row in enumerate(movie_df[9743:].itertuples()):
            try:
                item = reqs[index].json()
                movie_df.at[row.Index, "Title"] = item['Title'] 
                movie_df.at[row.Index, "Rated"] = item['Rated'] 
                movie_df.at[row.Index, "Released"] = item['Released'] 
                movie_df.at[row.Index, "Runtime"] = item['Runtime'] 
                movie_df.at[row.Index, "Genre"] = item['Genre'] 
                movie_df.at[row.Index, "Director"] = item['Director'] 
                movie_df.at[row.Index, "Writer"] = item['Writer'] 
                movie_df.at[row.Index, "Actors"] = item['Actors'] 
                movie_df.at[row.Index, "Plot"] = item['Plot'] 
                movie_df.at[row.Index, "Language"] = item['Language'] 
                movie_df.at[row.Index, "Country"] = item['Country'] 
                movie_df.at[row.Index, "Awards"] = item['Awards'] 
                movie_df.at[row.Index, "Poster"] = item['Poster'] 
                movie_df.at[row.Index, "Ratings"] = item['Ratings'] 
                movie_df.at[row.Index, "Metascore"] = item['Metascore'] 
                movie_df.at[row.Index, "imdbRating"] = item['imdbRating'] 
                movie_df.at[row.Index, "imdbVotes"] = item['imdbVotes'] 
                movie_df.at[row.Index, "imdbID"] = item['imdbID'] 
                movie_df.at[row.Index, "DVD"] = item['DVD'] 
                movie_df.at[row.Index, "BoxOffice"] = item['BoxOffice'] 
                movie_df.at[row.Index, "Production"] = item['Production'] 
                movie_df.at[row.Index, "Website"] = item['Website'] 

            except: 
                print("Record not found")
                

In [94]:
def exception_handler(request, exception):
    print("Request failed")

urls = ["http://www.omdbapi.com/?t={0}+&plot=full&apikey={1}".format(row[2].replace(" ", "+"), api_key) for row in movie_df[9743:].itertuples()]
rs = (grequests.get(u) for u in urls)
requests = grequests.map(rs, exception_handler=exception_handler, size=10)
appendData(requests)
pd.set_option('display.max_columns', None) 
movie_df.iloc[4750:4800]

Request failed
Request failed
Request failed
Request failed
Request failed
Request failed
Request failed
Request failed
Request failed
Request failed
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found
Record not found


Close                                   Movie Title O Theaters  Open  \
4750   5/30                                       Jason X       1878  4/26   
4751   6/13                                       Frailty       1497  4/12   
4752   6/20                                   Big Trouble       1961   4/5   
4753  11/28                                        Enigma         25  4/19   
4754   1/30                              The Fast Runner          35  4/12   
4755   6/20                                The Cat's Meow         11  4/12   
4756   7/18                                       Joshua          43  4/19   
4757   8/15                            Dogtown and Z-Boys         20  4/26   
4758  10/10                                   Nine Queens          5  4/19   
4759   7/18                                         Crush         10   4/5   
4760   6/13                                Savage Messiah         58  4/26   
4761   7/18                                The Salton Sea         15  4/26   
4762    5/9                                  Human Nature        224  4/12   
4763   9/26                                          Rain          1  4/26   
4764   9/12                           The Triumph of Love         18  4/17   
4765      -                                      Amadeus          18   4/5   
4766   5/30                               The Last Waltz           1   4/5   
4767   8/25                                       Circuit          3  4/26   
4768   9/26                                      Swimming          1   4/5   
4769  12/26                                 Les Destinees          4   4/5   
4770  11/24                               Murderous Maids          2  4/19   
4771   6/27                                 American Chai          5   4/5   
4772   3/20                                 The Komediant          2   4/5   
4773      -                                World Traveler          6  4/19   
4774  11/21                                          Sade          2  4/26   
4775      -                    The Earth Will Swallow You          5  4/12   
4776  11/21                              Girls Can't Swim          1  4/19   
4777    5/2                                  Spooky House         28  4/19   
4778   6/20                                 Chelsea Walls          3  4/19   
4779      -                                   Lucky Break          6   4/5   
4780      -                               New Best Friend        100  4/12   
4781   6/20                     Night at the Golden Eagle          3  4/26   
4782      -                                        Vulgar          2  4/26   
4783      -                                      Lansdown          1  4/26   
4784      -                                   Caravaggio           1  4/16   
4785      -                                     Some Body          1  4/26   
4786   8/18                                    Spider-Man       3615   5/3   
4787   11/3  Star Wars: Episode II - Attack of the Clones       3161  5/16   
4788    9/5                          The Sum of All Fears       3183  5/31   
4789   9/12              Spirit: Stallion of the Cimarron       3317  5/24   
4790   9/26                                      Insomnia       2610  5/24   
4791   8/13                                    Unfaithful       2617   5/8   
4792   7/18                                   About a Boy       1207  5/17   
4793   7/11                                        Enough       2623  5/24   
4794   7/18                            Undercover Brother       2167  5/31   
4795   6/30                                   The New Guy       2687  5/10   
4796   10/3               The Importance of Being Earnest         38  5/17   
4797   5/30                                   Deuces Wild       1480   5/3   
4798   6/27                              Hollywood Ending        765   5/3   
4799  11/26                            ESPN's Ultimate X          47  5/10   

           Opening Rank  St

In [3]:
actors_dfs = []
for page in range(1,4):
            one_df = pd.read_html(f"https://www.boxofficemojo.com/people/?view=Actor&pagenum={page}&sort=person&order=ASC&p=.htm", header=0)[2]
            one_df = one_df.rename(columns={ "Person(Click to view)": "Person", "# Movies / Average": "Number of Movies", 
                                        "# Movies / Average.1": "Average"})
            actors_dfs.append(one_df)
actors_df = pd.concat(actors_dfs)

In [4]:
actors_df = actors_df.reset_index(drop=True)
actors_df

Person Total Gross  Number of Movies Average  \
0                  Amy Adams    $2,402.9                30   $80.1   
1                Ben Affleck    $2,517.8                39   $64.6   
2              Casey Affleck      $922.8                24   $38.4   
3               Dianna Agron      $132.2                 5   $26.4   
4               Jessica Alba      $952.3                24   $39.7   
5                  Alan Alda      $753.0                22   $34.2   
6             Mahershala Ali    $1,399.7                13  $107.7   
7                 Joan Allen      $854.5                25   $34.2   
8                  Tim Allen    $1,763.0                20   $88.1   
9                Woody Allen      $461.3                27   $17.1   
10           Mathieu Amalric      $300.7                13   $23.1   
11          Gillian Anderson      $180.3                15   $12.0   
12             Julie Andrews      $929.0                14   $66.4   
13          Jennifer Aniston    $1,627.1                32   $50.8   
14               Aziz Ansari      $168.6                 3   $56.2   
15       Christina Applegate    $1,117.1                21   $53.2   
16                Tom Arnold      $415.9                19   $21.9   
17          Rosanna Arquette      $396.9                21   $18.9   
18            Gemma Arterton      $517.4                14   $37.0   
19               Kevin Bacon    $1,519.2                48   $31.7   
20              Penn Badgley      $134.6                 6   $22.4   
21              Alec Baldwin    $2,708.0                56   $48.4   
22            Christian Bale    $2,293.9                39   $58.8   
23                 Eric Bana    $1,068.3                17   $62.8   
24             Anne Bancroft      $548.0                28   $19.6   
25          Antonio Banderas    $2,386.4                43   $55.5   
26           Elizabeth Banks    $3,043.1                39   $78.0   
27             Javier Bardem      $725.0                19   $38.2   
28                Ben Barnes      $287.7                 5   $57.5   
29            Drew Barrymore    $1,614.6                36   $44.9   
..                       ...         ...               ...     ...   
840            Tom Wilkinson    $1,208.4                46   $26.3   
841      Michael K. Williams      $656.6                16   $41.0   
842        Michelle Williams    $1,156.4                31   $37.3   
843           Robin Williams    $3,279.3                50   $65.6   
844             Bruce Willis    $3,335.0                62   $53.8   
845              Luke Wilson      $656.7                32   $20.5   
846              Owen Wilson    $2,922.5                43   $68.0   
847           Patrick Wilson    $1,118.4                21   $53.3   
848             Rebel Wilson      $620.8                 9   $69.0   
849            Oprah Winfrey      $745.4                 9   $82.8   
850             Kate Winslet    $1,519.3                34   $44.7   
851  Mary Elizabeth Winstead      $480.0                18   $26.7   
852        Reese Witherspoon    $1,652.3                33   $50.1   
853            Finn Wolfhard      $334.3                 2  $167.1   
854               Ellen Wong       $31.5                 1   $31.5   
855              Elijah Wood    $1,889.3                37   $51.1   
856         Shailene Woodley      $614.7                 9   $68.3   
857            Kenny Wormald       $64.4                 4   $16.1   
858          Sam Worthington    $1,329.9                15   $88.7   
859           Jeffrey Wright    $2,006.7                27   $74.3   
860           Letitia Wright    $2,197.5                 6  $366.2   
861             Robin Wright      $837.2                 7  $119.6   
862            Anton Yelchin    $1,090.5                22   $49.6   
863               Donnie Yen      $804.7                14   $57.5   
864            Michelle Yeoh      $806.1                13   $62.0   
865              Steven Yeun    

In [5]:
for col in ['Total Gross','Gross', 'Average']:
    actors_df[col]= actors_df[col].replace({'\$': '',',': ''}, regex=True)
    actors_df[col]=[ float(x.replace('k', "")) * 1000 if 'k' in x else float(x)* 1000000  for x in actors_df[col] ]

actors_df

Person   Total Gross  Number of Movies      Average  \
0                  Amy Adams  2.402900e+09                30   80100000.0   
1                Ben Affleck  2.517800e+09                39   64600000.0   
2              Casey Affleck  9.228000e+08                24   38400000.0   
3               Dianna Agron  1.322000e+08                 5   26400000.0   
4               Jessica Alba  9.523000e+08                24   39700000.0   
5                  Alan Alda  7.530000e+08                22   34200000.0   
6             Mahershala Ali  1.399700e+09                13  107700000.0   
7                 Joan Allen  8.545000e+08                25   34200000.0   
8                  Tim Allen  1.763000e+09                20   88100000.0   
9                Woody Allen  4.613000e+08                27   17100000.0   
10           Mathieu Amalric  3.007000e+08                13   23100000.0   
11          Gillian Anderson  1.803000e+08                15   12000000.0   
12             Julie Andrews  9.290000e+08                14   66400000.0   
13          Jennifer Aniston  1.627100e+09                32   50800000.0   
14               Aziz Ansari  1.686000e+08                 3   56200000.0   
15       Christina Applegate  1.117100e+09                21   53200000.0   
16                Tom Arnold  4.159000e+08                19   21900000.0   
17          Rosanna Arquette  3.969000e+08                21   18900000.0   
18            Gemma Arterton  5.174000e+08                14   37000000.0   
19               Kevin Bacon  1.519200e+09                48   31700000.0   
20              Penn Badgley  1.346000e+08                 6   22400000.0   
21              Alec Baldwin  2.708000e+09                56   48400000.0   
22            Christian Bale  2.293900e+09                39   58800000.0   
23                 Eric Bana  1.068300e+09                17   62800000.0   
24             Anne Bancroft  5.480000e+08                28   19600000.0   
25          Antonio Banderas  2.386400e+09                43   55500000.0   
26           Elizabeth Banks  3.043100e+09                39   78000000.0   
27             Javier Bardem  7.250000e+08                19   38200000.0   
28                Ben Barnes  2.877000e+08                 5   57500000.0   
29            Drew Barrymore  1.614600e+09                36   44900000.0   
..                       ...           ...               ...          ...   
840            Tom Wilkinson  1.208400e+09                46   26300000.0   
841      Michael K. Williams  6.566000e+08                16   41000000.0   
842        Michelle Williams  1.156400e+09                31   37300000.0   
843           Robin Williams  3.279300e+09                50   65600000.0   
844             Bruce Willis  3.335000e+09                62   53800000.0   
845              Luke Wilson  6.567000e+08                32   20500000.0   
846              Owen Wilson  2.922500e+09                43   68000000.0   
847           Patrick Wilson  1.118400e+09                21   53300000.0   
848             Rebel Wilson  6.208000e+08                 9   69000000.0   
849            Oprah Winfrey  7.454000e+08                 9   82800000.0   
850             Kate Winslet  1.519300e+09                34   44700000.0   
851  Mary Elizabeth Winstead  4.800000e+08                18   26700000.0   
852        Reese Witherspoon  1.652300e+09                33   50100000.0   
853            Finn Wolfhard  3.343000e+08                 2  167100000.0   
854               Ellen Wong  3.150000e+07                 1   31500000.0   
855              Elijah Wood  1.889300e+09                37   51100000.0   
856         Shailene Woodley  6.147000e+08                 9   68300000.0   
857            Kenny Wormald  6.440000e+07                 4   16100000.0   
858          Sam Worthington  1.329900e+09                15   88700000.0   
859           Jeffrey Wright  2.006700e+09                27   74300000.0   
860           Leti

In [6]:
actors_df.dtypes

Person               object
Total Gross         float64
Number of Movies      int64
Average             float64
#1 Picture           object
Gross               float64
dtype: object

In [11]:
actors_df.to_csv("Actors_Data.csv", index=False)

In [95]:
movie_df.to_csv("Movies_Data_Updated.csv", index=False)